In [ ]:
# Install Ultralytics package
!pip install ultralytics


In [ ]:
# Check GPU
!nvidia-smi


In [ ]:
from ultralytics import YOLO

# Load YOLOv8 model (e.g., yolov8n.pt for YOLOv8 Nano)
model = YOLO('yolov8n.pt')

# Perform inference on an image
results = model('https://ultralytics.com/images/bus.jpg')

# Display results
results.show()


In [ ]:
# Mount Google Drive (if your dataset is stored there)
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Train YOLOv8 model on custom dataset
model.train(data='path/to/data.yaml', epochs=50, imgsz=640)


In [ ]:
# Export trained model to a desired format (e.g., ONNX, TorchScript)
model.export(format='onnx')


In [ ]:
# Load trained model
model = YOLO('path/to/trained_model.pt')

# Perform inference
results = model('path/to/test_image.jpg')

# Display results
results.show()


In [ ]:
# Clone YOLOv8 repository
!git clone https://github.com/ultralytics/ultralytics.git
%cd ultralytics
